In [19]:
import json
import os
import re
import translators as ts

In [20]:
# postman = json.decoder.JSONDecoder().decode(open('./Catea.postman_collection.json').read())
postman = json.decoder.JSONDecoder().decode(open('./Departement News Board.postman_collection.json').read())

In [21]:
# for idx_1,service in enumerate(postman["item"]):
#     for idx_2,request in enumerate(service["item"]):
#         postman["item"][idx_1]["item"][idx_2]["name"] = ts.google(request["name"], from_language="id", to_language="en")
#         for idx_3,response in enumerate(request["response"]):
#                 postman["item"][idx_1]["item"][idx_2]["response"][idx_3]["name"] = ts.google(response["name"], from_language="id", to_language="en")

In [22]:
# open('./Catea.postman_collection_translated.json', 'w').write(json.dumps(postman, indent=4))

In [23]:
def create_md(service, path):

    template_notepad_main = \
    """# {service_name} Service
{additional_notes}
{request_and_responses}
"""

    template_request = \
    """## Request: {request_name}
{method}<span style="padding:10px">**{url}**</span>\n
{request_body}
### Responses:
{responses}
<br>
"""

    template_request_body = \
    """```json
{request_body}
```
"""

    template_response = \
    """#### {response_name}
{response_status}
```json
{response_body}
```
"""    

    request_color_map = {
        "GET" : "green",
        "PUT" : "blue",
        "POST" : "yellow",
        "DELETE" : "red",
    }


    template_request_method = \
    """![{method}](https://badgen.net/badge/Method/{method}/{color})"""

    response_status_color_map = {
        "200" : "green",
        "201" : "green",
        "204" : "green",
        "400" : "red",
        "401" : "red",
        "403" : "red",
        "404" : "red",
        "500" : "red",
    }

    template_response_status = \
    """![{response_status}](https://badgen.net/badge/{response_status}/{response_code}/{color})"""

    additional_notes = """"""

    service_name = service["name"]
    service_name_lower = re.sub(r"[^a-zA-Z0-9]+", '-', service["name"].lower())

    request_str = ""

    with open(f"{path}README.md", "w") as f:
        for request in service["item"]:
            # for each request
            print(request["name"])
            request_name = request["name"]
            def_params  = []

            for idx, item in enumerate(request["request"]["url"]["path"]):
                if item.find(":") != -1:
                    typestr = ""
                    if item.find("id") != -1:
                        typestr = "int:"
                    else:
                        typestr = "string:"
                    item = item.replace(":", f"`<{typestr}")
                    item += ">`"
                def_params.append(item)

            request_url = "/" + "/".join(def_params)

            request_method = request["request"]["method"]
            request_method_str = template_request_method.format(method=request_method, color=request_color_map[request_method])

            request_body = ""

            if "body" in request["request"]:
                if request["request"]["body"]["raw"] != "":
                    request_body = template_request_body.format(request_body = json.dumps(json.decoder.JSONDecoder().decode(request["request"]["body"]["raw"]), indent=4).replace("\r", ""))

            response_str = ""

            for response in request["response"]:

                # for each response

                response_name = response["name"]
                response_code = response["code"]
                response_status = response["status"]
                response_status_str = template_response_status.format(response_code = response_code, response_status=response_status, color=response_status_color_map[str(response_code)])
                response_body = json.dumps(json.decoder.JSONDecoder().decode(response["body"]), indent=4)
                response_str += template_response.format(
                    response_name=response_name,
                    response_status=response_status_str,
                    response_body=response_body
                )

            request_str += template_request.format(
                request_name=request_name,
                method=request_method_str,
                url=request_url,
                request_body=request_body,
                responses=response_str
            )

        f.write(template_notepad_main.format(
                service_name=service_name,
                request_and_responses=request_str,
                additional_notes=additional_notes
            ))

        f.close()
    
    return template_notepad_main.format(
                service_name=service_name,
                request_and_responses=request_str,
                additional_notes=""
            )

def create_py(service, path):
    template_database_main_1 = \
"""from nameko.extensions import DependencyProvider
import mysql.connector
from mysql.connector import Error
import mysql.connector.pooling

class DatabaseWrapper:

    connection = None

    def __init__(self, connection):
        self.connection = connection

    ### CRUD TEMPLATE ###
    ### Feel free to remove or make changes ###

    # def get_all_room_type(self):
    #     cursor = self.connection.cursor(dictionary=True)
    #     result = []
    #     sql = \"\"\"
    #         SELECT * FROM room_type
    #     \"\"\"
    #     cursor.execute(sql)
    #     for row in cursor.fetchall():
    #         result.append({
    #             'id': row['id'],
    #             'name': row['name'],
    #             'price': row['price'],
    #             'capacity': row['capacity'],
    #             'status': row['status']
    #         })
    #     cursor.close()
    #     return result

    # def get_room_type_by_id(self, id):
    #     cursor = self.connection.cursor(dictionary=True)
    #     result = []
    #     sql = \"\"\"
    #         SELECT * FROM room_type WHERE id = {}
    #     \"\"\".format(id)
    #     cursor.execute(sql)
    #     result = cursor.fetchone()
    #     cursor.close()
    #     return result

    # def get_room_type_by_name(self, name):
    #     cursor = self.connection.cursor(dictionary=True)
    #     result = []
    #     sql = \"\"\"
    #         SELECT * FROM room_type WHERE name = '{}'
    #     \"\"\".format(name)
    #     cursor.execute(sql)
    #     result = cursor.fetchone()
    #     cursor.close()
    #     return result

    # def create_room_type(self, name, price, capacity):
    #     cursor = self.connection.cursor(dictionary=True)
    #     sql = \"\"\"
    #         INSERT INTO room_type (name, price, capacity, status)
    #         VALUES ('{}', {}, {}, 1)\"\"\".format(
    #         name, price, capacity)
    #     cursor.execute(sql)
    #     self.connection.commit()
    #     cursor.close()

    # def update_room_type(self, id, name, price, capacity, status):
    #     cursor = self.connection.cursor(dictionary=True)
    #     sql = \"\"\"
    #         UPDATE room_type
    #         SET name = '{}',
    #         price = {},
    #         capacity = {},
    #         status = {}
    #         WHERE id = {}\"\"\".format(
    #         name, price, capacity, status, id)
    #     cursor.execute(sql)
    #     self.connection.commit()
    #     cursor.close()

    # def delete_room_type(self, id):
    #     cursor = self.connection.cursor(dictionary=True)
    #     sql = \"\"\"
    #         DELETE FROM room_type
    #         WHERE id = {}\"\"\".format(id)
    #     cursor.execute(sql)
    #     self.connection.commit()
    #     cursor.close()
    
    ### START YOUR CODE HERE ###
    
    ### ...
    
    ### END YOUR CODE HERE ###
"""

    template_database_main_2 = \
"""class DatabaseProvider(DependencyProvider):

    connection_pool = None

    def setup(self):
        try:
            self.connection_pool = mysql.connector.pooling.MySQLConnectionPool(
                pool_name="database_pool",
                pool_size=32,
                pool_reset_session=True,
                host='127.0.0.1',
                database='catea_{service_name_u_s}',
                user='root',
                password=''
            )
        except Error as e:
            print("Error while connecting to MySQL using Connection pool ", e)

    def get_dependency(self, worker_ctx):
        return DatabaseWrapper(self.connection_pool.get_connection())
"""

    template_redis_main = \
"""import pickle
import uuid
import redis
from nameko.extensions import DependencyProvider

class SessionWrapper:
    def __init__(self, connection) -> None:
        self.redis = connection
        self.default_expiration = 60 * 60

    ### SESSION TEMPLATE ###
    ### Feel free to remove or make changes ###

    # def check_session_id(self, session_id):
    #     return self.redis.exists(session_id)

    # def generate_session_id(self):
    #     key = str(uuid.uuid4())
    #     while self.check_session_id(key):
    #         key = str(uuid.uuid4())
    #     self.redis.set(key, pickle.dumps({}))
    #     return key

    # def get_session_data(self, session_id):
    #     return pickle.loads(self.redis.get(session_id))

    # def set_session_data(self, session_id, data):
    #     self.redis.set(session_id, pickle.dumps(data))

    # def reset_session_data(self, session_id):
    #     self.redis.set(session_id, pickle.dumps({}))

    # def destroy_session_data(self, session_id):
    #     self.redis.delete(session_id)

    ### START YOUR CODE HERE ###
    
    ### ...
    
    ### END YOUR CODE HERE ###

class SessionProvider(DependencyProvider):
    def __init__(self):
        self.client = redis.Redis(host="127.0.0.1", port=6379, db=0)

    def get_dependency(self, worker_ctx):
        return SessionWrapper(self.client)
"""

    template_utils_main = \
"""import json
from werkzeug.wrappers import Response

def validate_request_body(request, request_body):
    for field in request_body:
        if field not in request.json:
            return Response(
                json.dumps({
                    'status': 'error',
                    'message': 'Missing required field: {}'.format(field)
                }),
                status=400,
                mimetype='application/json'
            )
        elif not request.json[field]:
            return Response(
                json.dumps({
                    'status': 'error',
                    'message': '{} cannot be empty'.format(field)
                }),
                status=400,
                mimetype='application/json'
            )
"""

    template_gateway_main = \
"""import json
from nameko.rpc import RpcProxy
from nameko.web.handlers import http
from werkzeug.wrappers import Response

class {service_name}GatewayService:
    name = '{service_name_u_s}_gateway'
    {service_name_u_s}_rpc = RpcProxy('{service_name_u_s}_service')
    ### BELOW IS AN AUTOMATICALLY GENERATED FUNCTION TEMPLATES ###
    {functions}
"""

    template_gateway_functions = \
"""
    @http('{method}', '{url}')
    def {function_name}(self, request{additional_path}):
        # PLEASE CODE WITH SPECIFICATION BELOW
        {request_body}

        ### RESPONSES
        {response_body}
        pass
"""

    template_request_body = \
"""

        ### REQUEST BODY
        # {request_body}
        """

    template_response_body= \
"""
        # {response_name}
        # BODY: 
        # {response_body}
        # STATUS CODE:
        # {response_status} | {response_code}
        """

    template_service_main = \
"""from nameko.rpc import rpc
import dependencies

class {service_name}Service:

    name = '{service_name_u_s}_service'

    database = dependencies.Database()
    
    ### CRUD TEMPLATE ###
    ### Feel free to remove or make changes ###

    # @rpc
    # def get_all_room_type(self):
    #     room_types = self.database.get_all_room_type()
    #     return room_types
    
    # @rpc
    # def get_room_type_by_id(self, id):
    #     room_type = self.database.get_room_type_by_id(id)
    #     return room_type
    
    # @rpc
    # def get_room_type_by_name(self, name):
    #     room_type = self.database.get_room_type_by_name(name)
    #     return room_type
    
    # @rpc
    # def create_room_type(self, name, price, capacity):
    #     self.database.create_room_type(name, price, capacity)
        
    # @rpc
    # def update_room_type(self, id, name, price, capacity, status):
    #     self.database.update_room_type(id, name, price, capacity, status)
        
    # @rpc
    # def delete_room_type(self, id):
    #     self.database.delete_room_type(id)
    
    ### START YOUR CODE HERE ###

    ### ...

    ### END YOUR CODE HERE ###
"""
    service_name = service["name"]
    service_name_u_s = re.sub(r"[^a-zA-Z0-9]+", '_', service["name"].lower())
    
    # create_md(service, f"./{service_name_lower}/")
    # create_md(service, f"")

    with open(f"{path}gateway.py", "w") as f:
        f2 = open(f"{path}service.py", "w")
        f3 = open(f"{path}utils.py", "w")
        f4 = open(f"{path}dependencies/redis.py", "w")
        f5 = open(f"{path}dependencies/database.py", "w")
        function_str = ""
        for request in service["item"]:


            # for each request
            request_name = request["name"]
            request_name_lower = request["name"]
            if request_name.find("(") != -1:
                request_name_lower = request_name[:request_name.find("(")]
            request_name_lower = ts.google(request_name_lower)
            request_name_lower = re.sub(r"[^a-zA-Z0-9]+", '_', request_name_lower.lower())

            additional_parameter = []

            def_params  = []

            for item in request["request"]["url"]["path"]:
                if item.find(":") != -1:
                    additional_parameter.append(item.replace(":", ""))
                    typestr = ""
                    if item.find("id") != -1:
                        typestr = "int:"
                    else:
                        typestr = "string:"
                    item = item.replace(":", f"<{typestr}")
                    item += ">"
                    
                def_params.append(item)

            additional_parameter = "" if len(additional_parameter) == 0 else f", {', '.join(additional_parameter)}"

            request_url = "/" + "/".join(def_params)
            request_method = request["request"]["method"]

            request_body = ""
            if "body" in request["request"]:
                if request["request"]["body"]["raw"] != "":
                    request_body = template_request_body.format(request_body = json.dumps(json.decoder.JSONDecoder().decode(request["request"]["body"]["raw"]), indent=4).replace("\r", "").replace("\n", "\n\t\t# "))

            response_str = ""

            for response in request["response"]:

                # for each response

                response_name = response["name"]
                response_code = response["code"]
                response_status = response["status"]
                response_body = response["body"]
                response_str += template_response_body.format(
                    response_name=response_name,
                    response_status=response_status,
                    response_code=response_code,
                    response_body=json.dumps(json.decoder.JSONDecoder().decode(response["body"]), indent=4).replace("\n", "\n\t\t# ")
                )

            function_str += template_gateway_functions.format(
                method=request_method,
                function_name = request_name_lower,
                additional_path = additional_parameter,
                url=request_url,
                request_body=request_body,
                response_body=response_str,
            )
        
        f.write(template_gateway_main.format(
                service_name=service_name.replace(" ",""),
                service_name_u_s=service_name_u_s,
                functions=function_str 
            ))
        f.close()
        
        f2.write(template_service_main.format(
                service_name=service_name.replace(" ",""),
                service_name_u_s=service_name_u_s,
            ))
        f2.close()
        

        f3.write(template_utils_main)
        f3.close()

        f4.write(template_redis_main)
        f4.close()

        f5.write(template_database_main_1 +template_database_main_2.format(
            service_name_u_s=service_name_u_s
        ))
        f5.close()

In [24]:
list_services  = postman["item"]

main_readme_str = ""

from tqdm.auto import tqdm
tqdm_bar = tqdm(total=len(list_services))
for service in list_services:
    # for each services
    service_name = re.sub(r"[^a-zA-Z0-9]+", '-', service["name"].lower())
    # # create folder
    folder = "./services/" + service_name
    # dpd_folder = "./services/" + service_name + "/dependencies"

    # if not os.path.exists(folder):
    #     os.makedirs(folder)
    # if not os.path.exists(dpd_folder):
    #     os.makedirs(dpd_folder)

    tqdm_bar.set_description("Creating markdown")
    main_readme_str += create_md(service, f"{folder}/")
    # tqdm_bar.set_description("Creating python files")
    # create_py(service, f"{folder}/")

    tqdm_bar.update(1)

with open("./README.md", "w") as f:
    f.write(main_readme_str)
    f.close()

  0%|          | 0/3 [00:00<?, ?it/s]

Register
Login
Logout
Get All News
Get News
Post a New News
Edit News
Delete News
Get File
